## XGBoost for District level MSW classifier

In [1]:
from xgboost import XGBClassifier

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Loading file
msw = pd.read_csv(r"C:\Users\AJain7\OneDrive - Stryker\Personal\Projects\Satellite Project\5. District_level\Cross-Sectional\anuj_anupam_scripts\2011_districts_10_bins.csv")

In [4]:
msw['MSW+AF8-2011'] = msw['MSW+AF8-2011'] -1

In [5]:
feature_cols = msw.columns[1:-6].tolist()

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

In [12]:
msw['MSW+AF8-2011'].value_counts()

1    279
2    254
0    107
Name: MSW+AF8-2011, dtype: int64

In [64]:
X_train, X_test, y_train, y_test = train_test_split(msw[feature_cols], msw['MSW+AF8-2011'], test_size=0.2, random_state=0)

In [65]:
xgbc = XGBClassifier(n_estimators=1000, max_depth=9, learning_rate=0.3,objective='binary:logistic', booster='gbtree', n_jobs=1, gamma=0, min_child_weight=6, max_delta_step=2, subsample=0.7, colsample_bytree=0.5, colsample_bylevel=0.9, colsample_bynode = 0.7, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5)

In [66]:
model = xgbc.fit(X_train, y_train)

In [67]:
predictions = model.predict(X_test)

In [68]:
f1_score(y_test, predictions, average='weighted')

0.8245069449324245

In [56]:
cv = KFold(n_splits=10, shuffle=True)

In [57]:
X = msw[feature_cols].values.copy()
y = msw['MSW+AF8-2011'].values.copy()

In [59]:
score = []
for train_index, test_index in cv.split(X):
    
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    best_xgb = xgbc.fit(X_train, y_train)
    
    predictions = best_xgb.predict(X_test)
    f1_weight = f1_score(y_test, predictions, average='weighted')
    
    score.append(f1_weight)

In [60]:
score

[0.7493239182692307,
 0.7819625334522746,
 0.7870978499686128,
 0.719859139307897,
 0.7464797529581069,
 0.8006715332868782,
 0.689709595959596,
 0.7033823287220027,
 0.8247906499556344,
 0.78125]

In [55]:
(0.7653573175725321 + 0.7653573175725321 + 0.7240538092440266 + 0.7173514484768342 + 0.7576603982300885)/5

0.7459560582192026

In [61]:
t = 0
for i in score:
    t= t+i

In [63]:
t/10

0.7584527301880234

In [1]:
import pickle

In [6]:
msltss = pickle.load(open('msltop_score_specs.pkl','rb'))
mslsco = pickle.load(open('mslscore_list.pkl','rb'))

In [60]:
print(msltss['xgBoost']['specs'][0])

{'n_estimators': 500, 'max_depth': 4, 'learning_rate': 0.25, 'verbosity': 1, 'objective': 'binary:hinge', 'booster': 'gbtree', 'n_jobs': -1, 'gamma': 0, 'min_child_weight': 6, 'max_delta_step': 4, 'subsample': 1, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.7, 'colsample_bynode': 0.9, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'base_score': 0.5, 'random_state': 0, 'importance_type': 'gain', 'n_randomized_search': 250, 'run_type': 'train-test', 'trail': 0, 'trail_type': 'sales', 'train_ratio': 0.9, 'val_train_loss': 'f1_weighted', 'top_n_models_drill': 5, 'top_n_models_pred': 5, 'n_drill_search': [50], 'test_ratio': 0.0, 'y_col': 'MSL+AF8-2011', 'cols_to_drop': [], 'ensemble': False, 'n_cols_dropped': 0, 'current_date_col': 'random_drop', 'no_steps_ahead': 0, 'time_cols': [], 'cross_validation': ['KFold'], 'kFold_splits': 5, 'pFold_splits': 10, 'feature_cols': ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21',

In [15]:
print(mslsco['xgBoost'][0]['train'])
print(mslsco['xgBoost'][0]['val'])

{'f1_weighted': 0.9665058483118112, 'f1_macro': 0.9666068702196541, 'f1_micro': 0.9665948275862069, 'accuracy_normal': 0.9665948275862069, 'accuracy_weighted': 0.9665231596619923, 'accuracy_classwise': array([0.98688525, 0.93548387, 0.97763578]), 'precision_weighted': 0.9667620573684236, 'precision_macro': 0.9668887502055609, 'precision_micro': 0.9665948275862069, 'recall_weighted': 0.9665948275862069, 'recall_macro': 0.9666682998723285, 'recall_micro': 0.9665948275862069}
{'f1_weighted': 0.9807234432234432, 'f1_macro': 0.9793650793650794, 'f1_micro': 0.9807692307692307, 'accuracy_normal': 0.9807692307692307, 'accuracy_weighted': 0.9829576690489281, 'accuracy_classwise': array([1.        , 1.        , 0.93548387]), 'precision_weighted': 0.9818376068376069, 'precision_macro': 0.9814814814814815, 'precision_micro': 0.9807692307692307, 'recall_weighted': 0.9807692307692307, 'recall_macro': 0.978494623655914, 'recall_micro': 0.9807692307692307}


In [17]:
emptss = pickle.load(open('emptop_score_specs.pkl','rb'))
empsco = pickle.load(open('empscore_list.pkl','rb'))

In [18]:
print(emptss['xgBoost']['scores'])

[0.7766120256164982, 0.7713457288776494, 0.7603426107287605, 0.7584945039250967, 0.752074843554807]


In [19]:
print(empsco['xgBoost'][0]['train'])
print(empsco['xgBoost'][0]['val'])

{'f1_weighted': 0.9571958603993317, 'f1_macro': 0.9572206134706134, 'f1_micro': 0.9571639586410635, 'accuracy_normal': 0.9571639586410635, 'accuracy_weighted': 0.957207841083876, 'accuracy_classwise': array([0.95089286, 0.96491228, 0.95555556]), 'precision_weighted': 0.9574002256318727, 'precision_macro': 0.9574927729636249, 'precision_micro': 0.9571639586410635, 'recall_weighted': 0.9571639586410635, 'recall_macro': 0.9571202311333891, 'recall_micro': 0.9571639586410635}
{'f1_weighted': 0.9605684885290148, 'f1_macro': 0.9604700854700855, 'f1_micro': 0.9605263157894737, 'accuracy_normal': 0.9605263157894737, 'accuracy_weighted': 0.9586349534643226, 'accuracy_classwise': array([0.92592593, 1.        , 0.96153846]), 'precision_weighted': 0.9626315789473684, 'precision_macro': 0.9605128205128205, 'precision_micro': 0.9605263157894737, 'recall_weighted': 0.9605263157894737, 'recall_macro': 0.9624881291547959, 'recall_micro': 0.9605263157894737}


In [20]:
bftss = pickle.load(open('bftop_score_specs.pkl','rb'))
bfsco = pickle.load(open('bfscore_list.pkl','rb'))

In [21]:
print(bftss['xgBoost']['scores'])

[0.8635347469631116, 0.8530616727665079, 0.8505711974498983, 0.8281565156931775, 0.8265409443911638]


In [22]:
print(bfsco['xgBoost'][0]['train'])
print(bfsco['xgBoost'][0]['val'])

{'f1_weighted': 0.9695747668321938, 'f1_macro': 0.969678024545395, 'f1_micro': 0.9695945945945946, 'accuracy_normal': 0.9695945945945946, 'accuracy_weighted': 0.969368197686358, 'accuracy_classwise': array([0.97923875, 0.95394737, 0.97627119]), 'precision_weighted': 0.9696321321321321, 'precision_macro': 0.9696145124716553, 'precision_micro': 0.9695945945945946, 'recall_weighted': 0.9695945945945946, 'recall_macro': 0.9698191030623301, 'recall_micro': 0.9695945945945946}
{'f1_weighted': 0.9797979797979798, 'f1_macro': 0.9818627450980393, 'f1_micro': 0.9797979797979798, 'accuracy_normal': 0.9797979797979798, 'accuracy_weighted': 0.9781129843241643, 'accuracy_classwise': array([0.975     , 1.        , 0.97058824]), 'precision_weighted': 0.9797979797979798, 'precision_macro': 0.9818627450980393, 'precision_micro': 0.9797979797979798, 'recall_weighted': 0.9797979797979798, 'recall_macro': 0.9818627450980393, 'recall_micro': 0.9797979797979798}


In [30]:
import pickle

In [31]:
assettss = pickle.load(open('assettop_score_specs.pkl','rb'))
# assetsco = pickle.load(open('assetscore_list.pkl','rb'))

In [34]:
assettss['xgBoost']['specs']

[{'n_estimators': 1000,
  'max_depth': 5,
  'learning_rate': 0.25,
  'verbosity': 1,
  'objective': 'binary:logistic',
  'booster': 'dart',
  'n_jobs': -1,
  'gamma': 0,
  'min_child_weight': 8,
  'max_delta_step': 10,
  'subsample': 0.5,
  'colsample_bytree': 1,
  'colsample_bylevel': 0.5,
  'colsample_bynode': 0.9,
  'reg_alpha': 0,
  'reg_lambda': 1,
  'scale_pos_weight': 1,
  'base_score': 0.5,
  'random_state': 0,
  'importance_type': 'weight',
  'n_randomized_search': 250,
  'run_type': 'train-test',
  'trail': 0,
  'trail_type': 'sales',
  'train_ratio': 0.9,
  'val_train_loss': 'f1_weighted',
  'top_n_models_drill': 5,
  'top_n_models_pred': 5,
  'n_drill_search': [50],
  'test_ratio': 0.0,
  'y_col': 'ASSET+AF8-2011',
  'cols_to_drop': [],
  'ensemble': False,
  'n_cols_dropped': 0,
  'current_date_col': 'random_drop',
  'no_steps_ahead': 0,
  'time_cols': [],
  'cross_validation': ['KFold'],
  'kFold_splits': 5,
  'pFold_splits': 4,
  'feature_cols': ['2',
   '3',
   '4',
   

In [23]:
mswtss = pickle.load(open('mswtop_score_specs.pkl','rb'))
mswsco = pickle.load(open('mswscore_list.pkl','rb'))

In [24]:
print(mswtss['xgBoost']['scores'])

[0.8471431418721289, 0.8399156231342844, 0.8263581346528153, 0.8195758404454887, 0.8090179037684025]


In [25]:
print(mswsco['xgBoost'][0]['train'])
print(mswsco['xgBoost'][0]['val'])

{'f1_weighted': 0.976081801910293, 'f1_macro': 0.9761077967065471, 'f1_micro': 0.9760956175298805, 'accuracy_normal': 0.9760956175298805, 'accuracy_weighted': 0.9759304410372495, 'accuracy_classwise': array([0.98770492, 0.97637795, 0.96470588]), 'precision_weighted': 0.976176890371223, 'precision_macro': 0.9760627737896245, 'precision_micro': 0.9760956175298805, 'recall_weighted': 0.9760956175298805, 'recall_macro': 0.976262917713878, 'recall_micro': 0.9760956175298805}
{'f1_weighted': 0.9763165266106443, 'f1_macro': 0.9768627450980393, 'f1_micro': 0.9761904761904762, 'accuracy_normal': 0.9761904761904762, 'accuracy_weighted': 0.9711459192085738, 'accuracy_classwise': array([0.94285714, 1.        , 1.        ]), 'precision_weighted': 0.9780219780219781, 'precision_macro': 0.9743589743589745, 'precision_micro': 0.9761904761904762, 'recall_weighted': 0.9761904761904762, 'recall_macro': 0.980952380952381, 'recall_micro': 0.9761904761904762}


In [100]:
fctss = pickle.load(open('fctop_score_specs.pkl','rb'))
# fcsco = pickle.load(open('fcscore_list.pkl','rb'))

In [101]:
print(fctss['xgBoost']['scores'])

[0.9085059504366132, 0.9026890888276583, 0.9008097045437132, 0.8953194207934893, 0.8953089501234059]


## Seeing scores 

In [2]:
import pickle

In [3]:
fctss = pickle.load(open('xgBoost-Results-AnujFeatures_V2/fctop_score_specs.pkl','rb'))
print(fctss['xgBoost']['val_scores'])
print(fctss['xgBoost']['train_scores'])
print('-------------')

[0.9097075251643565, 0.9039440186510163, 0.8919876588196788, 0.884326845945566, 0.8787304522890507]
[0.9991133181256053, 0.9991135682266796, 0.9991135680666469, 0.9988918436359064, 0.9988918861347248]
-------------


In [4]:
emptss = pickle.load(open('xgBoost-Results-AnujFeatures_V2/emptop_score_specs.pkl','rb'))
print(emptss['xgBoost']['val_scores'])
print(emptss['xgBoost']['train_scores'])
print('-------------')

[0.7784797811612577, 0.7395381756057515, 0.7353312026965003, 0.7347125582109633, 0.7268435826975391]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


In [5]:
mswtss = pickle.load(open('xgBoost-Results-AnujFeatures_V2/mswtop_score_specs.pkl','rb'))
print(mswtss['xgBoost']['val_scores'])
print(mswtss['xgBoost']['train_scores'])
print('-------------')

[0.8514848122495928, 0.8408969131855244, 0.8245183761318486, 0.8240534430714078, 0.823687215020459]
[0.9988050729660799, 0.9988051034803526, 0.9988055004876376, 0.9988050917409753, 0.9988050869571221]
-------------


In [6]:
msltss = pickle.load(open('xgBoost-Results-AnujFeatures_V2/msltop_score_specs.pkl','rb'))
print(msltss['xgBoost']['val_scores'])
print(msltss['xgBoost']['train_scores'])
print('-------------')

[0.8819755160675863, 0.8787414145488217, 0.8779687792804582, 0.8754972598686532, 0.8613770875276451]
[0.9990311575086602, 0.9990311602402541, 0.9990309006719293, 0.9990308787651958, 0.9990308723891366]
-------------


In [7]:
bftss = pickle.load(open('xgBoost-Results-AnujFeatures_V2/bftop_score_specs.pkl','rb'))
print(bftss['xgBoost']['val_scores'])
print(bftss['xgBoost']['train_scores'])
print('-------------')

[0.8494655101592123, 0.8419657697062671, 0.8399178620122308, 0.8379976955458635, 0.8289005862643762]
[0.998987033483516, 0.9989870053787044, 0.9989870180087722, 0.9989870158629518, 0.9989870287783308]
-------------


In [8]:
emp_femaletss = pickle.load(open('xgBoost-Results-AnujFeatures_V2/emp_femaletop_score_specs.pkl','rb'))
print(emp_femaletss['xgBoost']['val_scores'])
print(emp_femaletss['xgBoost']['train_scores'])
print('-------------')

[0.7401195411888216, 0.7248345257490144, 0.7105458596357251, 0.7043082789836379, 0.6973348478544594]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


In [9]:
empagnagtss = pickle.load(open('xgBoost-Results-AnujFeatures_V2/empagnagtop_score_specs.pkl','rb'))
print(empagnagtss['xgBoost']['val_scores'])
print(empagnagtss['xgBoost']['train_scores'])
print('-------------')

[0.729425381414981, 0.6955983202958134, 0.6855845607357874, 0.6744836628046299, 0.6656165661548364]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


In [120]:
littss = pickle.load(open('xgBoost-Results-AnujFeatures_V2/littop_score_specs.pkl','rb'))
print(littss['xgBoost']['val_scores'])
print(littss['xgBoost']['train_scores'])
print('-------------')

[0.814684413544797, 0.8102291562119802, 0.8044892404727847, 0.802840531940339, 0.7990877648090571]
[0.9990282353927304, 0.9990282267792555, 0.9990282502459534, 0.9990282342518648, 0.9990282407444606]
-------------


In [122]:
assettss = pickle.load(open('xgBoost-Results-AnujFeatures_V2/assettop_score_specs.pkl','rb'))
print(assettss['xgBoost']['val_scores'])
print(assettss['xgBoost']['train_scores'])
print('-------------')

[0.7994025198901287, 0.7897333701942448, 0.7879980248695724, 0.7816421027800107, 0.7749502022862697]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


In [3]:
chhtss = pickle.load(open('xgBoost-Results-AnujFeatures_V2/chhtop_score_specs.pkl','rb'))
print(chhtss['xgBoost']['val_scores'])
print(chhtss['xgBoost']['train_scores'])
print('-------------')

[0.7656011604849651, 0.7594576126509514, 0.755009503034964, 0.7453359267304812, 0.7428050316555641]
[0.9987421304933648, 0.9987421553869911, 0.998742124843343, 0.9987421588094968, 0.9987421313412584]
-------------


## Tesing on MSL-XGBoost 

In [4]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import numpy as np

def apply_smote(data, feature_cols, target):
    '''
    Input:
    data - the original dataframe
    feature_cols - the feature columns (list of columns)
    target - the target column (string value)
    '''
    sm = SMOTE(random_state=42)
    features, targets = sm.fit_resample(data[feature_cols],data[target])
    feature_df = pd.DataFrame(features, columns=feature_cols)
    target_df = pd.DataFrame(targets, columns=[target])
    output = pd.concat([feature_df, target_df], axis=1)
    # Shuffling dataset
    output = output.sample(frac=1).reset_index(drop=True)
    return output

filepath = '/Users/arpitjain/Downloads/SatPRo/2011_districts_10_bins_v2.csv'
data = pd.read_csv(filepath)
feature_cols = data.columns[1:-10].tolist()

Using TensorFlow backend.


In [5]:
data.columns

Index(['censuscode', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       'MSW+AF8-2011', 'BF+AF8-2011', 'MSL+AF8-2011', 'FC+AF8-2011',
       'CHH+AF8-2011', 'EMP+AF8-2011', 'ASSET+AF8-2011', 'EMP_FEMALE+AF8-2011',
       'LIT+AF8-2011', 'EMP_AG_NONAG+AF8-2011'],
      dtype='object', length=131)

In [6]:
data['CHH+AF8-2011'] = data['CHH+AF8-2011'] -1
y_col_CHH = 'CHH+AF8-2011'
output = apply_smote(data, feature_cols, y_col_CHH)
print(output['CHH+AF8-2011'].value_counts())
X = output[feature_cols].values
y = output[y_col_CHH].values

print(X.shape)
print(y.shape)

2    265
1    265
0    265
Name: CHH+AF8-2011, dtype: int64
(795, 120)
(795,)


In [7]:
chhtss['xgBoost']['specs'][0]

{'n_estimators': 500,
 'max_depth': 9,
 'learning_rate': 0.01,
 'verbosity': 1,
 'objective': 'binary:hinge',
 'booster': 'gbtree',
 'n_jobs': -1,
 'gamma': 0,
 'min_child_weight': 0,
 'max_delta_step': 4,
 'subsample': 0.7,
 'colsample_bytree': 0.7,
 'colsample_bylevel': 0.7,
 'colsample_bynode': 0.7,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'base_score': 0.5,
 'random_state': 0,
 'importance_type': 'cover',
 'n_randomized_search': 200,
 'run_type': 'train-test',
 'trail': 0,
 'trail_type': 'sales',
 'train_ratio': 0.9,
 'val_train_loss': 'f1_weighted',
 'top_n_models_drill': 5,
 'top_n_models_pred': 5,
 'n_drill_search': [50],
 'test_ratio': 0.0,
 'y_col': 'CHH+AF8-2011',
 'cols_to_drop': [],
 'ensemble': False,
 'n_cols_dropped': 0,
 'current_date_col': 'random_drop',
 'no_steps_ahead': 0,
 'time_cols': [],
 'cross_validation': ['KFold'],
 'kFold_splits': 5,
 'pFold_splits': 10,
 'feature_cols': ['1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',

In [8]:
n_estimators = chhtss['xgBoost']['specs'][0]['n_estimators']
max_depth = chhtss['xgBoost']['specs'][0]['max_depth']
learning_rate = chhtss['xgBoost']['specs'][0]['learning_rate']
objective = chhtss['xgBoost']['specs'][0]['objective']
booster = chhtss['xgBoost']['specs'][0]['booster']
gamma = chhtss['xgBoost']['specs'][0]['gamma']
min_child_weight = chhtss['xgBoost']['specs'][0]['min_child_weight']
max_delta_step = chhtss['xgBoost']['specs'][0]['max_delta_step']
subsample = chhtss['xgBoost']['specs'][0]['subsample']
colsample_bytree = chhtss['xgBoost']['specs'][0]['colsample_bytree']
colsample_bylevel = chhtss['xgBoost']['specs'][0]['colsample_bylevel']
colsample_bynode = chhtss['xgBoost']['specs'][0]['colsample_bynode']
reg_alpha = chhtss['xgBoost']['specs'][0]['reg_alpha']
reg_lambda = chhtss['xgBoost']['specs'][0]['reg_lambda']
scale_pos_weight = chhtss['xgBoost']['specs'][0]['scale_pos_weight']
base_score = chhtss['xgBoost']['specs'][0]['base_score']

In [9]:
xgbc = XGBClassifier(n_estimators=n_estimators, 
                     max_depth=max_depth, learning_rate=learning_rate, 
                     objective=objective, booster=booster,n_jobs=-1, 
                     gamma=gamma, min_child_weight=min_child_weight, 
                     max_delta_step=max_delta_step, subsample=subsample, 
                     colsample_bytree=colsample_bytree, 
                     colsample_bylevel=colsample_bylevel, colsample_bynode=colsample_bynode, 
                     reg_alpha=reg_alpha, reg_lambda=reg_lambda, 
                     scale_pos_weight=scale_pos_weight, base_score=base_score, random_state=0)

In [12]:
n_splits = 5
cv = KFold(n_splits=n_splits, shuffle=True)
val_score = []
train_score = []
for train_index, test_index in cv.split(X):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    best_xgb = xgbc.fit(X_train, y_train)
    
    predictions = best_xgb.predict(X_test)
    predictions_train = best_xgb.predict(X_train)
    f1_weight = f1_score(y_test, predictions, average='weighted')
    f1_weight_train = f1_score(y_train, predictions_train, average='weighted')
    
    val_score.append(f1_weight)
    train_score.append(f1_weight_train)

In [13]:
val_score = np.array(val_score)
print(val_score.mean())
train_score = np.array(train_score)
print(train_score.mean())

0.7717885906817592
0.9987421652524253


In [ ]:
# emp - 0.76
# emp_female - 0.74
# emp_agnag - 0.71
# asset - 0.78
# chh - 0.77

In [18]:
import pickle

## Phaneesh - seeing scores - bincount11

In [1]:
msltss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/msl_bincount11_top_score_specsPH.pkl','rb'))
print(msltss['xgBoost']['val_scores'])
print(msltss['xgBoost']['train_scores'])
print('-------------')

[0.8842873638866842, 0.864435017708602, 0.8601306992676572, 0.8591737776810675, 0.8535298391252784]
[0.9990610383959515, 0.9990610397621363, 0.9990610320953138, 0.9990610363084091, 0.9990610345371277]
-------------


In [2]:
mswtss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/msw_bincount11_top_score_specsPH.pkl','rb'))
print(mswtss['xgBoost']['val_scores'])
print('-------------')
print(mswtss['xgBoost']['train_scores'])

[0.8454350150918017, 0.8434906147785546, 0.8334341958176308, 0.8266183634459712, 0.8260913291845708]
-------------
[0.998801063087744, 0.9988010579583616, 0.9988010106381893, 0.9988010546896311, 0.9988010444349913]


In [3]:
fctss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/fc_bincount11_top_score_specsPH.pkl','rb'))
print(fctss['xgBoost']['val_scores'])
print(fctss['xgBoost']['train_scores'])
print('-------------')

[0.8256990528770427, 0.8252295031340431, 0.8175051960961206, 0.812222887326066, 0.8061414679518402]
[0.9990252639980259, 0.9990252988107461, 0.9985380439208311, 0.9985377096216466, 0.9990252635777985]
-------------


In [4]:
bftss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/bf_bincount11_top_score_specsPH.pkl','rb'))
print(bftss['xgBoost']['val_scores'])
print(bftss['xgBoost']['train_scores'])
print('-------------')

[0.8538163580362779, 0.8457065864664484, 0.843305895659588, 0.8235319746491019, 0.817825526565571]
[0.9990049782792662, 0.9990049771009227, 0.999004975313379, 0.9990049880412586, 0.9990049804639142]
-------------


In [9]:
assettss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/asset_bincount11_top_score_specsPH.pkl','rb'))
print(assettss['xgBoost']['val_scores'])
print(assettss['xgBoost']['train_scores'])
print('-------------')

[0.7721228027184247, 0.7682247188398207, 0.7641295128405168, 0.7624899395578504, 0.7596692023577104]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


In [10]:
chhtss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/chh_bincount11_top_score_specsPH.pkl','rb'))
print(chhtss['xgBoost']['val_scores'])
print(chhtss['xgBoost']['train_scores'])
print('-------------')

[0.7636069031469626, 0.7588481890394405, 0.7473855295369763, 0.7472581711351423, 0.7368419163888913]
[0.9987470624564103, 0.9987470591972267, 0.9987470552424016, 0.9987470679493586, 0.9987470391632656]
-------------


In [11]:
littss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/lit_bincount11_top_score_specsPH.pkl','rb'))
print(littss['xgBoost']['val_scores'])
print(littss['xgBoost']['train_scores'])
print('-------------')

[0.7893620004247651, 0.7867042727551119, 0.784608703912075, 0.7764017741089473, 0.7742881803842298]
[0.9990282541239457, 0.9958717626822, 0.9963596214095978, 0.9956287087298005, 0.9990282453544289]
-------------


In [12]:
empagnagtss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/emp_ag_nonag_bincount11_top_score_specsPH.pkl','rb'))
print(empagnagtss['xgBoost']['val_scores'])
print(empagnagtss['xgBoost']['train_scores'])
print('-------------')

[0.7065879840304155, 0.7039104098335612, 0.6972089465701918, 0.6867546336303603, 0.6825557416028044]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


In [13]:
empfemtss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/emp_female_bincount11_top_score_specsPH.pkl','rb'))
print(empfemtss['xgBoost']['val_scores'])
print(empfemtss['xgBoost']['train_scores'])
print('-------------')

[0.749914244850386, 0.7496650837250189, 0.7469845248207916, 0.7449008678893099, 0.735294916377369]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


In [2]:
emptss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/EMPtop_score_specs11.pkl','rb'))
print(emptss['xgBoost']['val_scores'])
print(emptss['xgBoost']['train_scores'])
print('-------------')

[0.7125023685905586, 0.7107955136369005, 0.7029738622876682, 0.6908744758032321, 0.6905891367929291]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


## Phannesh - seeing score - bincount 12 

In [14]:
msltss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/msl_bincount12_top_score_specsPH.pkl','rb'))
print(msltss['xgBoost']['val_scores'])
print(msltss['xgBoost']['train_scores'])
print('-------------')

[0.8731010688802282, 0.8668353753405607, 0.8623912105794235, 0.8618018821676328, 0.8507317086320633]
[0.9990610356148577, 0.999061033576216, 0.9990610352532538, 0.999295765034371, 0.9990610457255393]
-------------


In [15]:
mswtss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/msw_bincount12_top_score_specsPH.pkl','rb'))
print(mswtss['xgBoost']['val_scores'])
print(mswtss['xgBoost']['train_scores'])
print('-------------')

[0.8462333774597797, 0.844557552764494, 0.842583210685128, 0.8403723660419908, 0.8381036721216576]
[0.9988010542284617, 0.9988010750087046, 0.9988010587726887, 0.9988010562227092, 0.9988010512272961]
-------------


In [16]:
bftss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/bf_bincount12_top_score_specsPH.pkl','rb'))
print(bftss['xgBoost']['val_scores'])
print(bftss['xgBoost']['train_scores'])
print('-------------')

[0.8633574681543603, 0.8556962345302941, 0.8480904006486023, 0.8456014562843581, 0.8446340166837125]
[0.999004977705518, 0.9990049787866457, 0.9990049874980583, 0.9992537359506487, 0.9992537251362663]
-------------


In [17]:
fctss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/fc_bincount12_top_score_specsPH.pkl','rb'))
print(fctss['xgBoost']['val_scores'])
print(fctss['xgBoost']['train_scores'])
print('-------------')

[0.8399016694223794, 0.8319152383691444, 0.8256549705260934, 0.8195902266807227, 0.8182503805582355]
[0.9992688867626592, 0.9990253077887694, 0.9990252758931838, 0.9990252759011093, 0.999025274594987]
-------------


In [19]:
assettss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/asset_bincount12_top_score_specsPH.pkl','rb'))
print(assettss['xgBoost']['val_scores'])
print(assettss['xgBoost']['train_scores'])
print('-------------')

[0.7978223726304805, 0.7860844848389485, 0.7833823711321661, 0.7740709995848775, 0.7736208293797993]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


In [20]:
chhtss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/chh_bincount12_top_score_specsPH.pkl','rb'))
print(chhtss['xgBoost']['val_scores'])
print(chhtss['xgBoost']['train_scores'])
print('-------------')

[0.7538781778808298, 0.7475184135746569, 0.7430805660430961, 0.7268167708278435, 0.7198087596501768]
[0.9987470625780419, 0.9987470687055419, 0.9987470473110946, 0.998747055795415, 0.9987470629392787]
-------------


In [21]:
littss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/lit_bincount12_top_score_specsPH.pkl','rb'))
print(littss['xgBoost']['val_scores'])
print(littss['xgBoost']['train_scores'])
print('-------------')

[0.8119842436345021, 0.7972317925328899, 0.7913883528898905, 0.7880840066826224, 0.7810733780620076]
[0.9990282525218461, 0.9990282569204851, 0.9990282461129414, 0.9990282462154761, 0.9990282475516439]
-------------


In [22]:
emp_fem_tss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/emp_female_bincount12_top_score_specsPH.pkl','rb'))
print(emp_fem_tss['xgBoost']['val_scores'])
print(emp_fem_tss['xgBoost']['train_scores'])
print('-------------')

[0.7623081313395845, 0.7497838298456688, 0.7418063512210308, 0.7288201993438066, 0.7282592791950737]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


In [23]:
emp_ag_nonag_tss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/emp_ag_nonag_bincount12_top_score_specsPH.pkl','rb'))
print(emp_ag_nonag_tss['xgBoost']['val_scores'])
print(emp_ag_nonag_tss['xgBoost']['train_scores'])
print('-------------')

[0.7108897209725701, 0.7107653399331962, 0.7044358544642556, 0.7011656088621664, 0.6976780782410372]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


In [3]:
emptss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/EMPtop_score_specs12.pkl','rb'))
print(emptss['xgBoost']['val_scores'])
print(emptss['xgBoost']['train_scores'])
print('-------------')

[0.7130457746569906, 0.7060932641865885, 0.7051636936322592, 0.6991008319795704, 0.6866085407106246]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


In [4]:
emptss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/EMPtop_score_specs13.pkl','rb'))
print(emptss['xgBoost']['val_scores'])
print(emptss['xgBoost']['train_scores'])
print('-------------')

[0.7079294136201162, 0.7004921003126826, 0.7002396489798126, 0.6982328922455453, 0.6960376565321369]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


In [5]:
emptss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/EMPtop_score_specs14.pkl','rb'))
print(emptss['xgBoost']['val_scores'])
print(emptss['xgBoost']['train_scores'])
print('-------------')

[0.711310577829639, 0.7025015810585491, 0.7023656910828854, 0.6852391516153663, 0.673357552891615]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


## Phannesh - seeing score - bincount 16

In [2]:
import pickle

In [3]:
msltss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/MSLtop_score_specs16.pkl','rb'))
print(msltss['xgBoost']['val_scores'])
print(msltss['xgBoost']['train_scores'])
print('-------------')

[0.7703269417575981, 0.7510171854248392, 0.7419508294378021, 0.7400220074103276, 0.7355788155824128]
[0.9984355122989864, 0.9984355283138697, 0.998435508045387, 0.9984355059651089, 0.9984355080392614]
-------------


In [4]:
mswtss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/MSWtop_score_specs16.pkl','rb'))
print(mswtss['xgBoost']['val_scores'])
print(mswtss['xgBoost']['train_scores'])
print('-------------')

[0.7808832385427291, 0.7772470047737359, 0.7669486700469778, 0.7598110050535338, 0.7548422326069918]
[0.9984344093682779, 0.9984343402642969, 0.9984343818771888, 0.9988258021003162, 0.9984357309142888]
-------------


In [5]:
bftss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/BFtop_score_specs16.pkl','rb'))
print(bftss['xgBoost']['val_scores'])
print(bftss['xgBoost']['train_scores'])
print('-------------')

[0.7460061615345241, 0.7291911645721777, 0.7232178995307253, 0.7142522214411999, 0.7128390141448543]
[0.9984383940350229, 0.9984383953991219, 0.9984383691224734, 0.9984383981857929, 0.9984383785458076]
-------------


In [6]:
fctss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/FCtop_score_specs16.pkl','rb'))
print(fctss['xgBoost']['val_scores'])
print(fctss['xgBoost']['train_scores'])
print('-------------')

[0.7145017104013152, 0.6979451453697498, 0.694954875283657, 0.6891436827499142, 0.686843989553438]
[0.9984361691598729, 0.9984368225319772, 0.9984367680066818, 0.998436826868104, 0.9984361403820617]
-------------


In [7]:
assettss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/Assettop_score_specs16.pkl','rb'))
print(assettss['xgBoost']['val_scores'])
print(assettss['xgBoost']['train_scores'])
print('-------------')

[0.7078131504826599, 0.6908621917989585, 0.6879710591546401, 0.6842801938324816, 0.6715059592055699]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


In [8]:
chhtss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/CHHtop_score_specs16.pkl','rb'))
print(chhtss['xgBoost']['val_scores'])
print(chhtss['xgBoost']['train_scores'])
print('-------------')

[0.7006122731283227, 0.6929640911897557, 0.6857539750163851, 0.6816847290955325, 0.6709526710319623]
[0.998438003782875, 0.998438006226497, 0.9984380408426563, 0.9984380301086624, 0.998438010506684]
-------------


In [9]:
littss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/LITtop_score_specs16.pkl','rb'))
print(littss['xgBoost']['val_scores'])
print(littss['xgBoost']['train_scores'])
print('-------------')

[0.6441483398344082, 0.6399309530176238, 0.6360127198457451, 0.6169680972249978, 0.6150170435065145]
[0.9984390440258061, 0.9984390154979728, 0.9984390157716516, 0.9984390018353784, 0.9984390085741038]
-------------


In [10]:
emp_fem_tss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/EMP_FEMALEtop_score_specs16.pkl','rb'))
print(emp_fem_tss['xgBoost']['val_scores'])
print(emp_fem_tss['xgBoost']['train_scores'])
print('-------------')

[0.6991082196410804, 0.6827318489006833, 0.6783253104675367, 0.6713291174124555, 0.6623970798154121]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


In [12]:
emp_ag_nonag_tss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/EMP_AG_NONAGtop_score_specs16.pkl','rb'))
print(emp_ag_nonag_tss['xgBoost']['val_scores'])
print(emp_ag_nonag_tss['xgBoost']['train_scores'])
print('-------------')

[0.6486389528207818, 0.6312808137124821, 0.6226826114837541, 0.6147504648753251, 0.5910984830537162]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


## Phannesh - seeing score - bincount 17

In [13]:
import pickle

In [14]:
msltss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/MSLtop_score_specs17.pkl','rb'))
print(msltss['xgBoost']['val_scores'])
print(msltss['xgBoost']['train_scores'])
print('-------------')

[0.7417361856998091, 0.7353343054389929, 0.7346626667181277, 0.73277664858749, 0.7229949353828887]
[0.9984355039339909, 0.9984354854812624, 0.9984355971207897, 0.9984354796773299, 0.9984354941260744]
-------------


In [15]:
mswtss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/MSWtop_score_specs17.pkl','rb'))
print(mswtss['xgBoost']['val_scores'])
print(mswtss['xgBoost']['train_scores'])
print('-------------')

[0.7858541530918662, 0.7777806636858843, 0.7714685481373429, 0.7710144580280341, 0.7659074050630779]
[0.9984343244703107, 0.9984343279184984, 0.9984343264283255, 0.9984344376975057, 0.9984358521879884]
-------------


In [16]:
bftss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/BFtop_score_specs17.pkl','rb'))
print(bftss['xgBoost']['val_scores'])
print(bftss['xgBoost']['train_scores'])
print('-------------')

[0.7547797114482303, 0.734525332348897, 0.7154835166673648, 0.7142766266902898, 0.711621878658917]
[0.998438366559417, 0.9984383930127614, 0.9984383726298306, 0.9984383731406101, 0.9984383830222832]
-------------


In [17]:
fctss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/FCtop_score_specs17.pkl','rb'))
print(fctss['xgBoost']['val_scores'])
print(fctss['xgBoost']['train_scores'])
print('-------------')

[0.7080126192919262, 0.7002392352342558, 0.6964706788039521, 0.6917308296785076, 0.6830013126436351]
[0.9984368193436008, 0.9984368344267486, 0.9984369102295017, 0.9984368060824226, 0.9984368781847547]
-------------


In [18]:
assettss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/Assettop_score_specs17.pkl','rb'))
print(assettss['xgBoost']['val_scores'])
print(assettss['xgBoost']['train_scores'])
print('-------------')

[0.7105907148488282, 0.7039625229968496, 0.7026933042935636, 0.702576460097929, 0.6856277824942696]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


In [19]:
chhtss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/CHHtop_score_specs17.pkl','rb'))
print(chhtss['xgBoost']['val_scores'])
print(chhtss['xgBoost']['train_scores'])
print('-------------')

[0.7018914777933671, 0.6827072303338821, 0.6760629659612458, 0.6685700237493155, 0.662439153678957]
[0.9984380493413229, 0.9984380213281643, 0.9984380330370624, 0.9984380410116035, 0.9984380500319375]
-------------


In [20]:
littss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/LITtop_score_specs17.pkl','rb'))
print(littss['xgBoost']['val_scores'])
print(littss['xgBoost']['train_scores'])
print('-------------')

[0.6432906721312043, 0.6422298986101735, 0.6403119421337273, 0.6351315019579806, 0.62676358835652]
[0.9984389936696065, 0.9984390061828419, 0.99843901459721, 0.9984390163757564, 0.9984390171319021]
-------------


In [21]:
emp_fem_tss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/EMP_FEMALEtop_score_specs17.pkl','rb'))
print(emp_fem_tss['xgBoost']['val_scores'])
print(emp_fem_tss['xgBoost']['train_scores'])
print('-------------')

[0.7223454662343662, 0.7130904538840386, 0.6823682042777023, 0.6808956752557084, 0.673423850292674]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


In [22]:
emp_ag_nonag_tss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/EMP_AG_NONAGtop_score_specs17.pkl','rb'))
print(emp_ag_nonag_tss['xgBoost']['val_scores'])
print(emp_ag_nonag_tss['xgBoost']['train_scores'])
print('-------------')

[0.6381321403747149, 0.6305328869631411, 0.6280406289359506, 0.6206452987154873, 0.6061007417177273]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


In [1]:
emp_ag_nonag_tss = pickle.load(open('xgBoost-Results-PhaneeshFeatures_V1/EMP_AG_NONAGtop_score_specs17.pkl','rb'))
print(emp_ag_nonag_tss['xgBoost']['val_scores'])
print(emp_ag_nonag_tss['xgBoost']['train_scores'])
print('-------------')

[0.6381321403747149, 0.6305328869631411, 0.6280406289359506, 0.6206452987154873, 0.6061007417177273]
[1.0, 1.0, 1.0, 1.0, 1.0]
-------------


In [3]:
emp_ag_nonag_tss['xgBoost']['specs'][0]['kFold_splits']

5